In [182]:
from multiprocessing import Pool
from functools import partial
from __init__ import *

In [183]:
import Setting as _C
reload(_C)
import utils as _U
reload(_U)
import Simulation as sim
reload(sim)
import Dataloader as _D
reload(_D)

<module 'Dataloader' from 'c:\\Users\\shuai.song\\data\\Causis\\CTA_Trend\\Dataloader.py'>

In [3]:
## Multprocessing Usage Restrictions
if __name__ == "__main__":
    dirs = ['balance_sheet', 'features', 'trading_logs', 'v_analysis']

    for dir in dirs:
        if _C.VERSION not in os.listdir(f'output/{dir}'):
            os.mkdir(f'output/{dir}/{_C.VERSION}')

    pool = Pool(8)
    for i in range(len(_D.commodities.keys())):
        pool.apply_async(func=sim.wrapper_simulation, args=(list(_D.commodities.keys())[i], True, False)) # args: scom_, logger_, f_save
    
    pool.close()
    pool.join()

In [186]:
COM_D['CLOSE']>1

0       True
1       True
2       True
3       True
4       True
        ... 
3065    True
3066    True
3067    True
3068    True
3069    True
Name: CLOSE, Length: 3070, dtype: bool

In [184]:
sim.wrapper_simulation('p', True, True)

======= p:9/15 =======
Using Local 5min  Data | Local VTD: 2010-01-04--2022-08-19, BackTest VTD: 2010-01-04--2022-08-19
Using Local D     Data | Local VTD: 2010-01-04--2022-08-19, BackTest VTD: 2010-01-04--2022-08-19
Using Local 1min  Data | Local VTD: 2010-01-04--2022-08-19, BackTest VTD: 2010-01-04--2022-08-19


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [149]:
F_d = pd.read_pickle('dataset/F_data_1min.pkl')

In [150]:
F_d['rb']

,SYMBOL,CLOCK,OPEN,HIGH,LOW,CLOSE,VOLUME,INTEREST,SETTLE,ADJ,DATE
0,R.CN.SHF.rb.0004,2010-01-04 09:01:00,4523.0,4540.0,4523.0,4535.0,22172.0,914212.0,0.0,0.869811,2010-01-04
1,R.CN.SHF.rb.0004,2010-01-04 09:02:00,4535.0,4535.0,4524.0,4529.0,16936.0,914076.0,0.0,0.869811,2010-01-04
2,R.CN.SHF.rb.0004,2010-01-04 09:03:00,4528.0,4540.0,4528.0,4537.0,14964.0,914916.0,0.0,0.869811,2010-01-04
3,R.CN.SHF.rb.0004,2010-01-04 09:04:00,4537.0,4538.0,4529.0,4530.0,11020.0,918040.0,0.0,0.869811,2010-01-04
4,R.CN.SHF.rb.0004,2010-01-04 09:05:00,4530.0,4537.0,4529.0,4535.0,7766.0,919102.0,0.0,0.869811,2010-01-04
...,...,...,...,...,...,...,...,...,...,...,...
900505,R.CN.SHF.rb.0004,2022-08-19 22:56:00,3985.0,3988.0,3984.0,3986.0,4487.0,1492673.0,0.0,1.245742,2022-08-19
900506,R.CN.SHF.rb.0004,2022-08-19 22:57:00,3987.0,3987.0,3982.0,3983.0,3248.0,1492305.0,0.0,1.245742,2022-08-19
900507,R.CN.SHF.rb.0004,2022-08-19 22:58:00,3983.0,3987.0,3982.0,3984.0,3657.0,1491470.0,0.0,1.245742,2022-08-19
900508,R.CN.SHF.rb.0004,2022-08-19 22:59:00,3985.0,3988.0,3984.0,3988.0,5609.0,1489989.0,0.0,1.245742,2022-08-19


In [116]:
def Double_STD_trigger(stmp):
    tmp = 0.5*(2-np.abs(stmp['STD_S']+stmp['STD_S'].shift(1)))/stmp['STD_S']
    stmp['STD_S_Fork'] = tmp
    stmp['trigger'] = np.zeros(stmp.shape[0])
    
    buy_trigger_condition = (stmp['STD_S_Fork']== 1) & (stmp['STD_L']== 1)
    buy_observation_condition = (stmp['STD_S_Fork']==-1) & (stmp['STD_L']== 1)
    stmp['trigger'].loc[buy_trigger_condition] = 2
    stmp['trigger'].loc[buy_observation_condition] = 1

    sell_trigger_condition = (stmp['STD_S_Fork']==-1) & (stmp['STD_L']==-1)
    sell_observation_condition = (stmp['STD_S_Fork']== 1) & (stmp['STD_L']==-1)
    stmp['trigger'].loc[sell_trigger_condition] = -2
    stmp['trigger'].loc[sell_observation_condition] = -1
   
    return stmp

In [25]:
cc = Double_STD_trigger(cc)

In [124]:
## EMA Filter
def EMA_filter(_stmp_id, _stmp_d):
    # param: Day bars data
    _stmp_d['EMA_S'] = _stmp_d['CLOSE'].ewm(span=_C.DAY_EMA_S, adjust=False).mean()
    _stmp_d['EMA_L'] = _stmp_d['CLOSE'].ewm(span=_C.DAY_EMA_L, adjust=False).mean()
    Buy_signal_mask  = (_stmp_d['EMA_S']<_stmp_d['EMA_L']) & (_stmp_id['trigger'] >  0.5)
    Sell_signal_mask = (_stmp_d['EMA_S']>_stmp_d['EMA_L']) & (_stmp_id['trigger'] < -0.5)

    return Buy_signal_mask, Sell_signal_mask

def FLUCT_filter(_stmp):
    _stmp['FLUCT'] = _stmp['CLOSE'].ewm(span=15, min_periods=1).mean().rolling(30, min_periods=1).std()
    signal_mask = _stmp['FLUCT'] < _stmp['FLUCT'].rolling(300, min_periods=1).quantile(0.8)
    return signal_mask

In [125]:
## All in one Function
def Feature_and_Trigger(COM_ID, COM_D, filter, f_save):

    # COM_ID = _F.technical_analysis(COM_ID, PADJ=_C.PADJ, logger=False, save=f_save)
    COM_ID =  Double_STD_trigger(COM_ID)

    if len(filter):
        for f in filter:
            if f == 'EMA_filter':
                BSM_EMA, SSM_EMA = EMA_filter(COM_ID, COM_D)
                COM_ID['trigger'].loc[BSM_EMA] = 0.0
                COM_ID['trigger'].loc[SSM_EMA] = 0.0
            elif f == 'FLUCT_filter':
                SM = FLUCT_filter(COM_ID)
                COM_ID['trigger'].loc[~SM] = 0.0
            else:
                print('No Such Filter')

    return COM_ID, COM_D

In [126]:
COM_ID,  COM_D= Feature_and_Trigger(COM_ID, COM_D, _C.FILTER, False)

In [122]:
if len(_C.FILTER):
    for f in _C.FILTER:
        if f == 'EMA_filter':
            BSM_EMA, SSM_EMA = EMA_filter(COM_D)
            COM_ID['trigger'].loc[BSM_EMA] = 0.0
            COM_ID['trigger'].loc[SSM_EMA] = 0.0
        elif f == 'FLUCT_filter':
            SM = FLUCT_filter(COM_ID)
            COM_ID['trigger'].loc[~SM] = 0.0
        else:
            print('No Such Filter')

KeyError: 'trigger'

In [69]:
def Trading_trace(COM_ID, tick, COM_ID_VTD):
    ## trigger execution
    BUY_log = sim.BUY_Trade(COM_ID, tick, logger=True) 
    SELL_log = sim.SELL_Trade(COM_ID, tick, logger=True)

    buy = BUY_log.set_index('open_bar', drop=False)
    sell = SELL_log.set_index('open_bar', drop=False)

    trading_log_buy = buy.sort_index()
    trading_log_sell = sell.sort_index()

    trading_log_all = buy.append(sell)
    trading_log_all = trading_log_all.sort_index()

    ## patch some potential error
    trading_log_buy = sim.log_patch(trading_log_buy)
    trading_log_sell = sim.log_patch(trading_log_sell)
    trading_log_all = sim.log_patch(trading_log_all)

    ## create trade date columns
    def Date_Belong(clock):
        if clock[11:] < '21:00:00':
            return clock[:10]
        else:
            return COM_ID_VTD[min(COM_ID_VTD.index(clock[:10])+1, len(COM_ID_VTD)-1)]

    trading_log_buy['open_date'] = trading_log_buy['open_bar'].apply(lambda x: Date_Belong(x))
    trading_log_buy['close_date'] = trading_log_buy['close_bar'].apply(lambda x: Date_Belong(x))

    trading_log_sell['open_date'] = trading_log_sell['open_bar'].apply(lambda x: Date_Belong(x))
    trading_log_sell['close_date'] = trading_log_sell['close_bar'].apply(lambda x: Date_Belong(x))

    trading_log_all['open_date'] = trading_log_all['open_bar'].apply(lambda x: Date_Belong(x))
    trading_log_all['close_date'] = trading_log_all['close_bar'].apply(lambda x: Date_Belong(x))

    return [trading_log_buy, trading_log_sell, trading_log_all]

In [46]:
COM_ID = pd.read_csv('output/features/v3/p_features.csv', index_col=0)

In [58]:
COM_D = pd.read_pickle('dataset/F_data_D.pkl')

In [61]:
COM_ID = Double_STD_trigger(COM_ID)

In [77]:
COM_ID['trigger'].loc[COM_ID['trigger']==-2]

CLOCK
2010-01-04 09:06:00   -2.0
2010-01-04 09:24:00   -2.0
2010-01-04 09:45:00   -2.0
2010-01-04 14:08:00   -2.0
2010-01-04 14:21:00   -2.0
                      ... 
2022-08-18 22:19:00   -2.0
2022-08-19 10:00:00   -2.0
2022-08-19 11:06:00   -2.0
2022-08-19 13:56:00   -2.0
2022-08-19 14:14:00   -2.0
Name: trigger, Length: 23339, dtype: float64

In [70]:
logs = Trading_trace(COM_ID, 0, COM_ID['DATE'].drop_duplicates().to_list())

[1/23219]    | 2022-08-24 16:45:27 Done 
[11/23219]   | 2022-08-24 16:45:32 Done 
[21/23219]   | 2022-08-24 16:45:38 Done 
[31/23219]   | 2022-08-24 16:45:43 Done 


KeyboardInterrupt: 